In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
 
warnings.filterwarnings('ignore')

In [44]:
df= pd.read_csv('../../data/batdongsan/numerized/ChungCu_numerized-dataset.csv')
df = df.dropna()
print(df)


       Unnamed: 0  DienTich  \
1               1      50.0   
4               4     110.0   
6               6     105.5   
12             12      94.0   
15             15      92.0   
...           ...       ...   
22798       22798      73.0   
22812       22812     493.7   
22814       22814      57.0   
22816       22816      51.0   
22818       22818      80.0   

                                                  DiaChi  City  District  \
1      The Gold View, 346, Đường Bến Vân Đồn, Phường ...  79.0     773.0   
4      Dự án Seasons Avenue, Phường Mỗ Lao, Hà Đông, ...   1.0     268.0   
6      Dự án MHD Trung Văn, Đường Tố Hữu, Phường Trun...   1.0      19.0   
12     Dự án Resco Cổ Nhuế, Đường Phạm Văn Đồng, Phườ...   1.0      21.0   
15     Dự án Bình Minh Garden, Đường Đức Giang, Phườn...   1.0       4.0   
...                                                  ...   ...       ...   
22798  Dự án MT Eastmark City, Đường Vành Đai 3, Phườ...  79.0     763.0   
22812  Dự án Sunshi

In [45]:
# df = df.dropna(subset=['MucGia'])
df["USD"] = (df["MucGia"]/25000).round(2)
# df=df.drop(columns="MucGia")

In [46]:
df=df.drop(columns=["Unnamed: 0","DiaChi","MucGia","Lat","Long"])

In [47]:
df.head(5)

,DienTich,City,District,SoPhongNgu,SoToilet,NoiThat,HuongNha,HuongBanCong,PhapLy,Ward,USD
1,50.0,79.0,773.0,1.0,1.0,2.0,5.0,6.0,4.0,27298.0,148000.0
4,110.0,1.0,268.0,3.0,2.0,2.0,6.0,5.0,1.0,9541.0,220000.0
6,105.5,1.0,19.0,3.0,2.0,0.0,7.0,6.0,4.0,637.0,220000.0
12,94.0,1.0,21.0,3.0,2.0,2.0,1.0,2.0,1.0,617.0,146000.0
15,92.0,1.0,4.0,3.0,2.0,2.0,5.0,7.0,4.0,124.0,176000.0


In [48]:
# Assuming df is your DataFrame
X = df.iloc[:,0:10].values#features
y = df.iloc[:,10].values# Target variable

In [59]:
label_encoder = LabelEncoder()
x_numerical = df.select_dtypes(exclude=['object']).values
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
regressor = RandomForestRegressor(n_estimators=260, max_depth=100, random_state=0, oob_score=True)
# Fit the model
regressor.fit(X_train, y_train)
# Access the OOB Score
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')
# Making predictions on the test data
y_pred = regressor.predict(X_test)
# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

Out-of-Bag Score: 0.5576258770124625
Mean Squared Error: 8958526101.841373
Root Mean Squared Error: 94649.49076377206
R-squared: 0.7459651275732477


In [60]:
# Dự đoán trên tập huấn luyện
y_train_pred = regressor.predict(X_train)

# Đánh giá trên tập huấn luyện
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

print(f'Mean Squared Error (Train): {mse_train}')
print(f'Root Mean Squared Error (Train): {rmse_train}')
print(f'R-squared (Train): {r2_train}')


Mean Squared Error (Train): 3388093926.9024434
Root Mean Squared Error (Train): 58207.33567946951
R-squared (Train): 0.9443049910439668


# Random forest fill NA with KNN Imputer

sử dụng KNN Imputer - dựa trên các giá trị lân cận, ví dụ neighbors=2 với dữ liệu NA ở hàng 3 thì sẽ lấy cột 2 + cột 4 chia ra lấy trung bình

In [9]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Load data
df = pd.read_csv('../../data/batdongsan/numerized/ChungCu_numerized-dataset.csv')

# Kiểm tra các giá trị bị thiếu
print(df.isnull().sum())

# Khởi tạo KNN Imputer
imputer = KNNImputer(n_neighbors=5)

# Chọn các cột cần điền giá trị thiếu
columns_to_impute = df.select_dtypes(include=[np.number]).columns

# Điền giá trị thiếu
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

# Kiểm tra lại các giá trị bị thiếu
print(df.isnull().sum())

Unnamed: 0          0
DienTich            4
DiaChi              1
City                1
District          101
MucGia           2505
SoPhongNgu        595
SoToilet         1749
NoiThat          5313
Lat                34
Long               34
HuongNha        13649
HuongBanCong    12757
PhapLy           4492
Ward              867
dtype: int64
Unnamed: 0      0
DienTich        0
DiaChi          1
City            0
District        0
MucGia          0
SoPhongNgu      0
SoToilet        0
NoiThat         0
Lat             0
Long            0
HuongNha        0
HuongBanCong    0
PhapLy          0
Ward            0
dtype: int64


In [10]:
df = df.dropna()
print(df)

       Unnamed: 0  DienTich  \
0             0.0     50.00   
1             1.0     50.00   
2             2.0     73.00   
3             3.0     33.24   
4             4.0    110.00   
...           ...       ...   
22822     22822.0     68.00   
22823     22823.0     53.00   
22824     22824.0     46.00   
22825     22825.0     51.00   
22826     22826.0     63.60   

                                                  DiaChi  City  District  \
0      Dự án D-Aqua, Đường Bến Bình Đông, Phường 14, ...  79.0     776.0   
1      The Gold View, 346, Đường Bến Vân Đồn, Phường ...  79.0     773.0   
2      The Gloria by Silk Path, số 8, Đường Nguyên Hồ...   1.0       6.0   
3      Dự án Ben Hill Thuận An , Đường Thuận Giao 24,...  74.0     725.0   
4      Dự án Seasons Avenue, Phường Mỗ Lao, Hà Đông, ...   1.0     268.0   
...                                                  ...   ...       ...   
22822  Dự án Saigon Asiana, Đường Nguyễn Văn Luông, P...  79.0     775.0   
22823  Dự án The Dr

In [11]:
# df = df.dropna(subset=['MucGia'])
df["USD"] = (df["MucGia"]/25000).round(2)
# df=df.drop(columns="MucGia")

In [12]:
df=df.drop(columns=["Unnamed: 0","DiaChi","MucGia","Lat","Long"])

In [13]:
df.head(5)

,DienTich,City,District,SoPhongNgu,SoToilet,NoiThat,HuongNha,HuongBanCong,PhapLy,Ward,USD
0,50.00,79.0,776.0,1.0,1.0,3.0,5.4,6.0,2.4,27421.0,90000.0
1,50.00,79.0,773.0,1.0,1.0,2.0,5.0,6.0,4.0,27298.0,148000.0
2,73.00,1.0,6.0,2.0,2.0,2.0,4.0,3.0,1.0,199.0,24313600.0
3,33.24,74.0,725.0,1.0,1.8,2.6,4.6,5.8,1.8,25972.0,40000.0
4,110.00,1.0,268.0,3.0,2.0,2.0,6.0,5.0,1.0,9541.0,220000.0


In [14]:
# Assuming df is your DataFrame
X = df.iloc[:,0:10].values#features
y = df.iloc[:,10].values# Target variable# Target variable

In [15]:
label_encoder = LabelEncoder()
x_numerical = df.select_dtypes(exclude=['object']).values
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0, oob_score=True)

# Fit the model
regressor.fit(X_train, y_train)

# Access the OOB Score
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

# Making predictions on the test data
y_pred = regressor.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')



Out-of-Bag Score: 0.10265887466767099
Mean Squared Error: 49317553637249.125
Root Mean Squared Error: 7022645.77187609
R-squared: 0.12336938420252641


In [16]:
# Dự đoán trên tập huấn luyện
y_train_pred = regressor.predict(X_train)

# Đánh giá trên tập huấn luyện
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

print(f'Mean Squared Error (Train): {mse_train}')
print(f'Root Mean Squared Error (Train): {rmse_train}')
print(f'R-squared (Train): {r2_train}')


Mean Squared Error (Train): 38636192906331.75
Root Mean Squared Error (Train): 6215801.871547367
R-squared (Train): 0.2920236102335232


# Random forest fill NA with median value

In [17]:

from sklearn.impute import SimpleImputer
# Load data
df = pd.read_csv('../../data/batdongsan/numerized/ChungCu_numerized-dataset.csv')

# Kiểm tra các giá trị bị thiếu
print(df.isnull().sum())
# Khởi tạo SimpleImputer
imputer = SimpleImputer(strategy='mean')
columns_to_round = ['SoToilet', 'SoPhongNgu', 'NoiThat', 'HuongNha', 'HuongBanCong', 'PhapLy']

# Điền giá trị thiếu
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])
df[columns_to_round] = df[columns_to_round].round()

# Kiểm tra lại các giá trị bị thiếu
print(df.isnull().sum())

# Các bước tiếp theo tương tự như trên


Unnamed: 0          0
DienTich            4
DiaChi              1
City                1
District          101
MucGia           2505
SoPhongNgu        595
SoToilet         1749
NoiThat          5313
Lat                34
Long               34
HuongNha        13649
HuongBanCong    12757
PhapLy           4492
Ward              867
dtype: int64
Unnamed: 0      0
DienTich        0
DiaChi          1
City            0
District        0
MucGia          0
SoPhongNgu      0
SoToilet        0
NoiThat         0
Lat             0
Long            0
HuongNha        0
HuongBanCong    0
PhapLy          0
Ward            0
dtype: int64


In [18]:
df = df.dropna()
print(df)

       Unnamed: 0  DienTich  \
0             0.0     50.00   
1             1.0     50.00   
2             2.0     73.00   
3             3.0     33.24   
4             4.0    110.00   
...           ...       ...   
22822     22822.0     68.00   
22823     22823.0     53.00   
22824     22824.0     46.00   
22825     22825.0     51.00   
22826     22826.0     63.60   

                                                  DiaChi  City  District  \
0      Dự án D-Aqua, Đường Bến Bình Đông, Phường 14, ...  79.0     776.0   
1      The Gold View, 346, Đường Bến Vân Đồn, Phường ...  79.0     773.0   
2      The Gloria by Silk Path, số 8, Đường Nguyên Hồ...   1.0       6.0   
3      Dự án Ben Hill Thuận An , Đường Thuận Giao 24,...  74.0     725.0   
4      Dự án Seasons Avenue, Phường Mỗ Lao, Hà Đông, ...   1.0     268.0   
...                                                  ...   ...       ...   
22822  Dự án Saigon Asiana, Đường Nguyễn Văn Luông, P...  79.0     775.0   
22823  Dự án The Dr

In [19]:
# df = df.dropna(subset=['MucGia'])
df["USD"] = (df["MucGia"]/25000).round(2)
# df=df.drop(columns="MucGia")

In [20]:
df=df.drop(columns=["Unnamed: 0","DiaChi","MucGia","Lat","Long"])

In [21]:
df.head(5)

,DienTich,City,District,SoPhongNgu,SoToilet,NoiThat,HuongNha,HuongBanCong,PhapLy,Ward,USD
0,50.00,79.0,776.0,1.0,1.0,3.0,6.0,6.0,2.0,27421.0,90000.00
1,50.00,79.0,773.0,1.0,1.0,2.0,5.0,6.0,4.0,27298.0,148000.00
2,73.00,1.0,6.0,2.0,2.0,2.0,4.0,3.0,1.0,199.0,225130.61
3,33.24,74.0,725.0,1.0,2.0,2.0,6.0,6.0,2.0,25972.0,40000.00
4,110.00,1.0,268.0,3.0,2.0,2.0,6.0,5.0,1.0,9541.0,220000.00


In [22]:
# Assuming df is your DataFrame
X = df.iloc[:,0:10].values#features
y = df.iloc[:,10].values# Target variable# Target variable

In [23]:
label_encoder = LabelEncoder()
x_numerical = df.select_dtypes(exclude=['object']).values
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0, oob_score=True)

# Fit the model
regressor.fit(X_train, y_train)

# Access the OOB Score
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

# Making predictions on the test data
y_pred = regressor.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')



Out-of-Bag Score: -0.14366069180205865
Mean Squared Error: 169967377005.89474
Root Mean Squared Error: 412270.99947230675
R-squared: -3.0521514115478423
